# Выбор региона для разработки новых нефтяных месторождений

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.


#### Часть 1. [Загрузка и подготовка данных](#part1)
* [1. Импорт библиотек.](#part1.1)
* [2. Загрузка датасетов](#part1.2)

#### Часть 2. [Обучение и проверка модели](#part2)
* [1. Разбиение данных на обучающую и валидационную выборки](#part2.1)
* [2. Обучение модели.](#part2.2)

#### Часть 3. [Подготовка к расчёту прибыли](#part3)

#### Часть 4. [Расчет прибыли и рисков](#part4)

#### Часть 5. [Общий вывод](#part5)


<a id='part1'></a>
# 1. Загрузка и подготовка данных
<a id='part1.1'></a>
## 1.1 Импорт билиотек

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st
from sklearn.utils import shuffle

<a id='part1ю2'></a>
## 1.2 Загрузка датасетов

In [2]:
data=[]
for i in range (0,3):
    data.append(pd.read_csv('/datasets/geo_data_'+str(i)+'.csv'))
    data[i].info()
display(data[0].head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


## Вывод
Библиотеки импортированы, данные загружены.
Пропусков нет, предобработка не требуется.

<a id='part2'></a>
# 2. Обучение и проверка модели
<a id='part2.1'></a>
## 2.1 Разбиение данных на обучающую и валидационную выборки

In [3]:
features=[]
target=[]
features_train=[]
features_valid=[]
target_train=[]
target_valid=[]

for i in range (0,3):
    
    target.append(data[i]['product'])
    features.append(data[i].drop(['product','id'], axis=1))
    
    features_train_i, features_valid_i, target_train_i, target_valid_i = train_test_split(
    features[i], target[i], test_size=0.25, random_state=1)
    
    features_train.append(features_train_i.reset_index(drop=True))
    features_valid.append(features_valid_i.reset_index(drop=True))
    target_train.append(target_train_i.reset_index(drop=True))
    target_valid.append(target_valid_i.reset_index(drop=True))

<a id='part2.2'></a>
## 2.2 Обучение модели

In [4]:
predicted_valid=[]
for i in range(0,3):
    model = LinearRegression()
    model.fit(features_train[i], target_train[i])
    predicted_valid_i = model.predict(features_valid[i])
    predicted_valid.append(predicted_valid_i)
    print('Регион {}\n Средний предсказанный запас: {:.1f} тыс. баррелей'.format(i, pd.Series(predicted_valid_i).mean()))
    print('RMSE: {:.1f}\n'.format(mean_squared_error(predicted_valid_i,target_valid[i])**0.5))

Регион 0
 Средний предсказанный запас: 92.5 тыс. баррелей
RMSE: 37.7

Регион 1
 Средний предсказанный запас: 69.1 тыс. баррелей
RMSE: 0.9

Регион 2
 Средний предсказанный запас: 95.0 тыс. баррелей
RMSE: 39.9



## Вывод
Данные разбиты на обучающую и валидационную выборки в соотношение 75:25.  
Модель линейной регрессии обучена.  
Средний предсказанный запас выше всего в регионах 0 и 2 с разницей в 2.5 тысячи баррелей.  
В регионе 1 самая маленькая среднеквадратичная ошибка, что означает, что в нем получилась самая точная модель, но и средние предсказанные запасы самые маленькие.

<a id='part3'></a>
# 3. Подготовка к расчёту прибыли

In [5]:
# Бюджет на разработку 10 млрд
budget = 10000000000
# Доход с 1 тыс баррелей 45 тыс
income = 450000
# Кол-во разрабатываемых скважин
wells = 200
# Сколько тыс баррелей надо для достижения безубыточности
total_amount = budget/income
# Сколько в среднем должно быть запасов в одной скважине
mean_amount_per_well = total_amount/wells
print('Для безубыточной разработки надо добыть {:.1f} тыс. баррелей'.format(total_amount))
print('При разработке 200 скважин средний объем скважины должен быть {:.1f} тыс. баррелей'.format(mean_amount_per_well))

Для безубыточной разработки надо добыть 22222.2 тыс. баррелей
При разработке 200 скважин средний объем скважины должен быть 111.1 тыс. баррелей


In [6]:
# Функция для расчета прибыли
# Выбирает n скважин с наибольшими запасами из смоделированных данных
# и возвращает полученную прибыль (суммарный доход - бюджет) из реальных данных
def revenue(target,predicts,n,total_budget,income_per_well):

    predicts_sorted = pd.Series(predicts).sort_values(ascending=False)
    targets_selected = target[predicts_sorted.index].head(n).sum()

    return targets_selected*income_per_well - total_budget

for i in range(0,3):
    print('Прибыль в регионе',i,'при разработке 200 скважин с наибольшими запасами')
    print('{:.1f}\n'.format(revenue(target_valid[i],predicted_valid[i],wells,budget,income)))

Прибыль в регионе 0 при разработке 200 скважин с наибольшими запасами
3260781417.8

Прибыль в регионе 1 при разработке 200 скважин с наибольшими запасами
2415086696.7

Прибыль в регионе 2 при разработке 200 скважин с наибольшими запасами
2563093352.2



## Вывод
В каждом регионе средний запас скважины меньше запаса, необходимого для безубыточной разработки. Следовательно, нужно воспользоваться бутстрепом для определения наиболее прибыльного региона.  
Для этого была написана функция для расчета прибыли от выбранного количества скважин.  
Функция была проверена на 200 скважинах каждого региона с наибольшими запасами.

<a id='part4'></a>
# 4. Расчёт прибыли и рисков 

In [7]:
state = np.random.RandomState(1)
values1=[]
for i in range(0,3):
    values = []
    count=0
    # бутстпреп
    for j in range(1000):
        # по условию выбираем 500 случайных скважин
        target_subsample = target_valid[i].sample(n=500,replace=True, random_state=state)
        # находим по индексам соответствующие им предсказанные данные
        predicted_subsample = pd.Series(predicted_valid[i])[target_subsample.index]
        # считаем прибыль
        values.append(revenue(target_subsample, predicted_subsample, wells,budget,income))
        # для расчета рисков. Если прибыль отрицательная, счетчик +1
        if revenue(target_subsample, predicted_subsample,wells,budget,income) < 0:
            count+=1
    
    
    values = pd.Series(values)
    print('Регион', i)
    print('Средняя прибыль: {:.1f}'.format(values.mean()))
    
    #Расчет доверительного интервала
    alpha=0.95
    df=len(values)-1
    loc=values.mean()
    scale=values.sem()
    confidence_interval = st.t.interval(alpha,df,loc,scale)
    print("95%-ый доверительный интервал: {:.1f}, {:.1f}".format(confidence_interval[0],confidence_interval[1]))
    print('Дельта: {:.1f}'.format(confidence_interval[1]-confidence_interval[0]))
    print('Риск убытков:',count/10,'%\n')    

Регион 0
Средняя прибыль: 464377670.4
95%-ый доверительный интервал: 447036498.1, 481718842.6
Дельта: 34682344.5
Риск убытков: 5.2 %

Регион 1
Средняя прибыль: 541636803.4
95%-ый доверительный интервал: 528122791.9, 555150814.9
Дельта: 27028023.0
Риск убытков: 0.4 %

Регион 2
Средняя прибыль: 420170371.3
95%-ый доверительный интервал: 402746771.3, 437593971.4
Дельта: 34847200.1
Риск убытков: 6.7 %



## Вывод
Проведен бутстреп для каждого региона (1000 раз было выбрано 500 скважин, далее для 200 скважин с наибольшими запасами была посчитана средняя прибыль, 95-% доверительный интервал и риск убытков.

<a id='part5'></a>
# Общий вывод

В результате работы были изучены данные и обучены модели для предсказания запасов в скважинах.

С помощью процедуры Bootstrap для 1000 выборок были найдены распределения прибыли для каждого региона для 200 из 500 случайно выбранных скважин.

В итоге, в регионе 1, несмотря на наименьшие средние предсказанные запасы, оказалась самая высокая средняя прибыль (541 млн), наименьший риск убытков (0.4%) и самый маленький разброс внутри доверительного интервала (27 млн).

Таким образом, наиболее перспективным регионом для разработки является регион 1.